In [1]:
from gmail import Gmail
from datetime import datetime
from setfit import SetFitModel
from bs4 import BeautifulSoup
import os
import pandas as pd
from flan import JobTitleCompanyNameExtractor
# from huggingchat import HFJobTitleCompanyNameExtractor
from tqdm import tqdm




In [2]:
email_clasification_model = SetFitModel.from_pretrained("./../models/job_email_classification_model_final")
job_stage_classification_model = SetFitModel.from_pretrained("./../models/job_stage_classification_model")
extractor = JobTitleCompanyNameExtractor()
def infer_one_sentence(model, sentence):
        predtext = [sentence]
        predicted_class = model(predtext)
        return str(predicted_class.numpy()[0])

/Users/akshit/Library/Caches/pypoetry/virtualenvs/talenttrove-mo4SAvxM-py3.10/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/akshit/Library/Caches/pypoetry/virtualenvs/talenttrove-mo4SAvxM-py3.10/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [11]:
def get_email_csv():
    gmail = Gmail(username="akshit.karanam99@gmail.com",password="uknf iafz dpna qyjb")
    gmail.authenticate()
    specified_date = datetime(2023, 11, 1)
    formatted_date = specified_date.strftime("%d-%b-%Y")
    ids = gmail.get_email_by_date(from_date=formatted_date)
    email_dict = gmail.parse_emails(ids[:10])
    emails_final_list = []
    for email in tqdm(email_dict):
        try:
            subject = (email['subject']).decode('utf-8')
        except:
            subject = email['subject']

        html = str(BeautifulSoup(email['body']).text)
        string_list = [s.strip() for s in str(html).split()]
        final_string = ' '.join(string_list)
        final_string_without_subject = ' '.join(string_list)
        final_string = "Subject: " + str(subject) +". Body: " + final_string
        stage = ""
        predicted_class = infer_one_sentence(email_clasification_model,final_string_without_subject)
        if predicted_class == "1":
            stage = infer_one_sentence(model=job_stage_classification_model,sentence=final_string)
            company = extractor.get_company(final_string)
            title = extractor.get_jobtitle(final_string)
            print(stage)
            print(company)
            print(title)
            emails_final_list.append({'id':email['id'] , "date": email['date'],'title': title,"company":company,'stage':stage})
    df = pd.DataFrame(emails_final_list)        

    file_path = './../../data/applied_data/jobs_applied.csv'

    if os.path.exists(file_path):

        df.to_csv(file_path, mode='a', index=False, header=False)
    else:
        if df.empty:
            df = pd.DataFrame(columns=['id','date','title','company','stage'])
        df.to_csv(file_path, index=False)    

In [12]:
get_email_csv()

 40%|████      | 4/10 [00:01<00:01,  4.01it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (5470 > 512). Running this sequence through the model will result in indexing errors
/Users/akshit/Library/Caches/pypoetry/virtualenvs/talenttrove-mo4SAvxM-py3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 50%|█████     | 5/10 [00:43<01:16, 15.38s/it]

3
Linkedin - Career Opportunities for Data Scientists - Career Opportunity: Data Scientist in
Linkedin - Career Opportunities for Data Scientists - Career Opportunity: Data Scientist in


/Users/akshit/Library/Caches/pypoetry/virtualenvs/talenttrove-mo4SAvxM-py3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 60%|██████    | 6/10 [01:16<01:26, 21.52s/it]

3
Linkedin's job alert for "software engineer": Akshit Karanam (
Linkedin's job alert for "software engineer": Akshit Karanam (


/Users/akshit/Library/Caches/pypoetry/virtualenvs/talenttrove-mo4SAvxM-py3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 80%|████████  | 8/10 [01:32<00:26, 13.44s/it]

2
MATCHLAH LLP
MATCHLAH LLP


 90%|█████████ | 9/10 [01:32<00:09,  9.32s/it]/Users/akshit/Library/Caches/pypoetry/virtualenvs/talenttrove-mo4SAvxM-py3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 10/10 [01:34<00:00,  9.41s/it]

2
Hello! We are here to help!
Hi there, Are you stuck and need some help? We noticed you haven't imported
